In [1]:
import torch
import torch.nn as nn
import lightning

In [ ]:
input_image = torch.rand(3,11)
print(input_image.size())

In [ ]:
model = nn.Sequential(
    nn.Linear(11, 20),
    nn.ReLU(),
    nn.Linear(20, 10),
    nn.ReLU(),
    nn.Linear(10, 3),
    nn.ReLU(),
    nn.Linear(3, 1),
)
model(input_image)	

# actual test

In [2]:
from vessel_dataloader import VesselDataset
import lightning as L
from pathlib import Path

DATA_DIR = Path(r"C:\Users\Eric Kaufmann\workspace\MA\data\carotid_flow_database")

dataset = VesselDataset(DATA_DIR)
dataset[0][0].shape, dataset[0][1].shape

(torch.Size([10001, 3]), torch.Size([3]))

In [3]:
from torch import optim, nn

basic_model = nn.Sequential(
    nn.Linear(10001, 20),
    nn.ReLU(),
    nn.Linear(20, 10),
    nn.ReLU(),
    nn.Linear(10, 3),
    nn.ReLU(),
    nn.Linear(3, 1),
)

class BasicModel(L.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def training_step(self, batch):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        x = x.T
        y = y.view(-1, 1)

        x_hat = self.model(x)
        loss = nn.functional.mse_loss(x_hat, y)
        # Logging to TensorBoard (if installed) by default
        #self.log("train_loss", loss)
        self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=True, batch_size=1)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
m = BasicModel(basic_model)

In [4]:
%reload_ext tensorboard
%tensorboard --logdir=src/lightning_logs/
batch_size = 1

trainer = L.Trainer(limit_train_batches=1000, max_epochs=5)
trainer.fit(model=m, train_dataloaders=dataset)

Reusing TensorBoard on port 6007 (pid 19992), started 0:03:03 ago. (Use '!kill 19992' to kill it.)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 200 K 
-------------------------------------
200 K     Trainable params
0         Non-trainable params
200 K     Total params
0.801     Total estimated model params size (MB)


Epoch 1:  71%|███████   | 93/131 [00:15<00:06,  5.87it/s, v_num=11, train_loss_step=0.191, train_loss_epoch=0.244] 